---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [3]:
import pandas as pd
import numpy as np

energy=pd.ExcelFile("Energy Indicators.xls").parse("Energy",skiprows=10,
                                                       usecols=range(2,6)).dropna()
energy.columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
energy['Country']=energy['Country'].apply(lambda temp:str(temp).split("(")[0].strip())
energy['Country']=energy['Country'].str.rstrip('1234567890')
energy['Energy Supply']=energy["Energy Supply"].replace({'...':np.NaN})
energy['Energy Supply per Capita']=energy["Energy Supply per Capita"].replace({'...':np.NaN})
energy["Energy Supply"]*=1000000
energy['Country']=energy['Country'].replace({"Republic of Korea": "South Korea",
                                            "United States of America": "United States",
                                            "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                                            "China, Hong Kong Special Administrative Region": "Hong Kong"})
energy=energy.set_index("Country")

gdp=pd.read_csv("world_bank.csv",skiprows=4)
gdp=gdp.rename(columns={"Country Name":"Country"})
gdp["Country"]=gdp['Country'].replace({"Korea, Rep.": "South Korea", 
                                    "Iran, Islamic Rep.": "Iran",
                                    "Hong Kong SAR, China": "Hong Kong"})
gdp=gdp.sort_values(by=["Country"])
gdp=gdp.set_index("Country")
gdp=gdp[gdp.columns[-10:]]

ScimEn=pd.ExcelFile("scimagojr-3.xlsx").parse("Sheet1").dropna()
ScimEn = ScimEn.set_index("Country")

def merge(df1,df2):
    return pd.merge(df1,df2,how="inner",left_index=True,right_index=True)

def outer_merge(df1,df2):
    return pd.merge(df1,df2,how="outer",left_index=True,right_index=True)

def answer_one():    
    Top15 = ScimEn.reset_index().sort_values(by=["Country"])
    Top15 = ScimEn.iloc[0:15]
    
    Top15 = merge(Top15, energy)
    return merge(Top15, gdp)

gdp

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,
Afghanistan,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
Albania,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10
Algeria,1.446474e+11,1.495654e+11,1.531550e+11,1.556055e+11,1.612073e+11,1.658823e+11,1.715223e+11,1.763249e+11,1.830252e+11,1.901632e+11
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
Angola,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
Antigua and Barbuda,1.268714e+09,1.389229e+09,1.390217e+09,1.222890e+09,1.135539e+09,1.115174e+09,1.159999e+09,1.155982e+09,1.211723e+09,1.256122e+09
Arab World,1.763824e+12,1.863481e+12,1.980983e+12,2.011001e+12,2.103825e+12,2.173896e+12,2.318267e+12,2.385580e+12,2.436659e+12,2.509068e+12
Argentina,3.788140e+11,4.089887e+11,4.215649e+11,4.217758e+11,4.616402e+11,5.003555e+11,5.043671e+11,5.189199e+11,5.212737e+11,NaN


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [47]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [94]:
def mergelen(df1,df2):
    return len(merge(df1,df2))

def answer_two():
    energy_gdp=mergelen(energy,gdp)
    energy_scimen=mergelen(ScimEn,energy)
    gdp_scimen=mergelen(gdp,ScimEn)
    
    energy_gdp_scimen=mergelen(merge(energy,gdp),ScimEn)
    
    energy_U_gdp_U_scimen = len(outer_merge(outer_merge(energy,gdp),ScimEn))
    
    return energy_U_gdp_U_scimen - energy_gdp_scimen

answer_two()

156

### Question 3 (6.6%)
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [49]:
def add_column(df,name,function_to_use,list_columns):
    df[name] = df[list_columns].apply(function_to_use, axis=1)
    return df
    
def sort_column(df,name):
    return df.sort_values(by=[name],ascending=False)

def calculate_avgGDP(df):
    last_10_years = [str(x) for x in range(2006,2016)]
    
    df = add_column(df,"avgGDP",np.mean,last_10_years)
    return sort_column(df,"avgGDP")

def answer_three():
    Top15 = answer_one()
    return calculate_avgGDP(Top15)["avgGDP"]

answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGDP, dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [95]:
def answer_four():
    Top15 = answer_one()
    last_10_years=[str(x) for x in range(2006,2016)]
    
    Top15 = calculate_avgGDP(Top15)
    
    sixth_largest=Top15.iloc[5]
    return sixth_largest["2015"] - sixth_largest["2006"]

# answer_four()

### Question 5 (6.6%)
What is the mean energy supply per capita?

*This function should return a single number.*

In [96]:
def answer_five():
    Top15 = answer_one()
    return Top15["Energy Supply per Capita"].mean()

# answer_five()

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [52]:
def index_sort(df, column_to_be_added, keep_existing_index=True, ascOrder=False):
    list_columns = [column_to_be_added]
    if keep_existing_index == True:
        list_columns.append(df.index)
    else:
        df = df.reset_index()
    return df.set_index(list_columns).sort_index(ascending=ascOrder)

def index_sort_and_argmax(df, column_to_be_added, position=0):
    df = index_sort(df, column_to_be_added, )
    return reverse(df.index[position])

def reverse(index):
    return index[::-1]

def answer_six():
    Top15 = answer_one()
    return index_sort_and_argmax(Top15,"% Renewable")

answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [53]:
def ratio(df, col_A, col_B):
    return df[col_A] / df[col_B]

def answer_seven():
    Top15 = answer_one()
    Top15["% Self Citations"] = ratio(Top15, "Self-citations", "Citations")
    return index_sort_and_argmax(Top15, "% Self Citations")

answer_seven()

('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [54]:
def answer_eight():
    Top15 = answer_one()
    Top15["Population"] = ratio(Top15, "Energy Supply", "Energy Supply per Capita")
    return index_sort_and_argmax(Top15, "Population", 2)[0]

answer_eight()

'United States'

### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [55]:
def calculate_Population(df,population="Population"):
    df[population] = ratio(df, "Energy Supply", "Energy Supply per Capita")
    return df

def answer_nine():
    Top15 = answer_one()
    Top15 = calculate_Population(Top15)
    Top15["Citable docs per Capita"] = ratio(Top15, "Citable documents", "Population")
    return Top15.corr(method='pearson', min_periods=1)["Citable docs per Capita"]["Energy Supply per Capita"]

answer_nine()

0.79400104354429435

In [56]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [57]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [69]:
def answer_ten():
    Top15 = answer_one()
    Top15["HighRenew"] = 0
    Top15.loc[Top15["% Renewable"] >= Top15["% Renewable"].median(),"HighRenew"] = 1
    
    include_index, ascending = False, True
    return index_sort(Top15, "Rank", include_index, ascending).set_index("Country")["HighRenew"]

answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [97]:
def calculate_Continent(df):
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    set_of_continents = sorted(set(ContinentDict.values()))
    geo_df = pd.DataFrame(list(ContinentDict.values()),index=ContinentDict.keys()).rename(columns={0:"Continent"})
    
    geo_df["Continent"].astype('category', 
                                    categories = set_of_continents,
                                    ordered = True)
    df["Continent"]=geo_df.loc[df.index]
    return df

def index_and_group(df, index):
    return df.reset_index().set_index(index).groupby(level=0)
    
def answer_eleven():
    Top15 = answer_one()
    Top15 = calculate_Population(Top15)
    Top15 = calculate_Continent(Top15)
    Top15 = index_and_group(Top15,"Continent")["Population"].agg({'size':np.size,
                                                                             'sum':np.sum,
                                                                             'mean':np.mean,
                                                                             'std':np.std,})
    return Top15[["size","sum","mean","std"]]

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [85]:
def answer_twelve():
    Top15 = answer_one()
    Top15 = calculate_Continent(Top15)
    Top15["% Renewable"] = pd.cut(Top15['% Renewable'], 5)
    Top15 = Top15.reset_index().set_index(["Continent","% Renewable"]).groupby(level=[0,1])["Country"].agg({'size':np.size})
    return Top15["size"]

answer_twelve()

Continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
Name: size, dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [86]:
def answer_thirteen():
    Top15 = answer_one()
    Top15 = calculate_Population(Top15, "PopEst")
    Top15["PopEst"] = Top15["PopEst"].apply(lambda x:"{:,}".format(x))
    return Top15["PopEst"]

answer_thirteen()

Country
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Canada                  35,239,864.86486486
China                 1,367,645,161.2903225
France                  63,837,349.39759036
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
Iran                    77,075,630.25210084
Italy                  59,908,256.880733944
Japan                  127,409,395.97315437
Russian Federation            143,500,000.0
South Korea            49,805,429.864253394
Spain                    46,443,396.2264151
United Kingdom         63,870,967.741935484
United States          317,615,384.61538464
Name: PopEst, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [87]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [88]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!